# Solucion 3 - Deep Learning

Con esta solución queremos probar solamente que tan bien se ajustan algunos modelos de redes neuronales utilizando el estado del arte en deep learning utilizando TensorFlow y/o Keras.

## Cargamos librerías a usar y los datos

In [5]:
%matplotlib inline

from time import time
from itertools import product
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.metrics import r2_score, make_scorer, accuracy_score, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale, StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample

from IPython.display import display, HTML

plt.style.use('seaborn-white')

In [6]:
# Cargamos los datos completos para seleccionar mas variables
ataques_train = pd.read_csv('data/kddcup.data_clean.csv', sep=',', decimal='.')
print("Cantidad de observaciones %i con %i variables: " %(ataques_train.shape[0],ataques_train.shape[1]))
feature_cols = ['same_srv_rate', 'flag_SF', 'dst_host_same_srv_rate', 'service_private',
       'dst_host_srv_serror_rate', 'service_http', 'logged_in',
       'dst_host_srv_count', 'count', 'srv_serror_rate', 'flag_S0',
       'dst_host_serror_rate', 'dst_host_count', 'rerror_rate', 'serror_rate',
       'dst_host_rerror_rate', 'src_bytes', 'srv_rerror_rate',
       'dst_host_same_src_port_rate', 'dst_host_srv_rerror_rate',
       'protocol_type_udp', 'service_ecr_i', 'flag_REJ', 'service_pop_3',
       'protocol_type_tcp', 'diff_srv_rate', 'hot', 'dst_host_diff_srv_rate',
       'service_telnet', 'service_domain_u', 'wrong_fragment',
       'dst_host_srv_diff_host_rate', 'num_compromised', 'service_smtp',
       'srv_count', 'dst_bytes', 'srv_diff_host_rate', 'service_ftp_data',
       'duration', 'service_ftp', 'attack_category']

ataques_train = ataques_train[feature_cols]
# Cargamos los datos de validación del 10% de la competencia 
ataques_10prec_test = pd.read_csv('data/data_10per_test_preprocessed.csv', sep=',', decimal='.')
ataques_test = ataques_10prec_test[feature_cols]

print("Cantidad de observaciones %i con %i variables (Entrenamiento) " %(ataques_train.shape[0],ataques_train.shape[1]))
print("Cantidad de observaciones %i con %i variables (Validación) " %(ataques_test.shape[0],ataques_test.shape[1]))

# Balanceamos los datos
df_normal = ataques_train[ataques_train.attack_category=='normal']
df_dos = ataques_train[ataques_train.attack_category=='dos']
df_probe = ataques_train[ataques_train.attack_category=='probe']
df_r2l = ataques_train[ataques_train.attack_category=='r2l']
df_u2r = ataques_train[ataques_train.attack_category=='u2r']
#df_unknown = ataques_train[ataques_train.attack_category=='unknown']

# Remuestreo tomando solo un conjnto de datos menor en las clases de mayor frecuencia
df_normal_downsampled = resample(df_normal, replace=False, n_samples=200000, random_state=123)
df_dos_downsampled = resample(df_dos, replace=False, n_samples=100000, random_state=123)
# Combinar las clases con los nuevos datos remuestreados
ataques_train = pd.concat([df_normal_downsampled, df_dos_downsampled, df_probe, df_r2l, df_u2r])

# Eliminamos los datos "unknown" de la muestra de validación, los cuales no tenemos como entrenar
ataques_test = ataques_test[ataques_test.attack_category!='unknown']

# Mostrar las cantidades de los nuevos datos
print("Balanceo de datos: ")
print(ataques_train.attack_category.value_counts())
print(ataques_test.attack_category.value_counts())

# Definimos los datos en entrenamiento y validación
X = ataques_train.drop(['attack_category'], axis=1)
y = ataques_train.attack_category.copy()
X_test_40var = ataques_test.drop(['attack_category'], axis=1)
y_test_40var = ataques_test.attack_category.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=35)

Cantidad de observaciones 1074992 con 122 variables: 
Cantidad de observaciones 1074992 con 41 variables (Entrenamiento) 
Cantidad de observaciones 311029 con 41 variables (Validación) 
Balanceo de datos: 
normal    200000
dos       100000
probe      13860
r2l          999
u2r           52
Name: attack_category, dtype: int64
dos       223298
normal     60593
r2l         5993
probe       2377
u2r           39
Name: attack_category, dtype: int64


## Multilayer Perceptron (scikit-learn)